## CHATBOT named Chat-o-fy

In [1]:
# Import necessary libraries
import tkinter as tk
from tkinter import messagebox, scrolledtext
from PIL import Image, ImageTk, ImageDraw
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import logging

In [2]:
# Set up logging
logging.basicConfig(level=logging.DEBUG)

In [3]:
# Load the data
data_path = "Conversation.csv"
df = pd.read_csv(data_path)

In [4]:
df

,serial,question,answer
0,0,"hi, how are you doing?",i'm fine. how about yourself?
1,1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,3,no problem. so how have you been?,i've been great. what about you?
4,4,i've been great. what about you?,i've been good. i'm in school right now.
...,...,...,...
3720,3720,that's a good question. maybe it's not old age.,are you right-handed?
3721,3721,are you right-handed?,yes. all my life.
3722,3722,yes. all my life.,you're wearing out your right hand. stop using...
3723,3723,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.


In [5]:
# Ensure the data contains expected columns
assert 'question' in df.columns
assert 'answer' in df.columns

In [6]:
df

,serial,question,answer
0,0,"hi, how are you doing?",i'm fine. how about yourself?
1,1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,3,no problem. so how have you been?,i've been great. what about you?
4,4,i've been great. what about you?,i've been good. i'm in school right now.
...,...,...,...
3720,3720,that's a good question. maybe it's not old age.,are you right-handed?
3721,3721,are you right-handed?,yes. all my life.
3722,3722,yes. all my life.,you're wearing out your right hand. stop using...
3723,3723,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.


In [7]:
# Remove any rows with missing values
df.dropna(subset=['question', 'answer'], inplace=True)

In [8]:
# Split the data into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=4)

In [9]:
# Train the model
def train_model(df):
    # Define a pipeline for text classification
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_df=0.85, min_df=2)),
        ('classifier', MultinomialNB())
    ])
    
    # Hyperparameter tuning using GridSearchCV
    param_grid = {
        'tfidf__max_df': [0.85, 0.9, 0.95],
        'tfidf__min_df': [1, 2, 5],
        'classifier__alpha': [0.1, 0.5, 1.0]
    }
    
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
    grid_search.fit(df['question'], df['answer'])
    
    logging.debug(f"Best Parameters: {grid_search.best_params_}")
    logging.debug(f"Best Score: {grid_search.best_score_}")
    
    return grid_search.best_estimator_

In [10]:
pipeline = train_model(train_df)

C:\Users\sukal\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
DEBUG:root:Best Parameters: {'classifier__alpha': 0.1, 'tfidf__max_df': 0.85, 'tfidf__min_df': 5}
DEBUG:root:Best Score: 0.0087248322147651


In [11]:
test_predictions = pipeline.predict(test_df['question'])

In [12]:
test_predictions

array(['why not?', 'yes, i did.', 'but a cop might stop you.', 'why not?',
       'why not?', 'what do you mean?', 'what do you mean?',
       'yes, i did.', "but you're doing 75.", 'why is that?',
       'of course.', 'really? why is it?', 'why not?',
       'what do you mean?', 'what do you mean?', 'what do you mean?',
       "no, i'm serious.", 'what do you mean?', 'what do you mean?',
       'why not?', 'what do you mean?', 'of course.', "i don't know.",
       'of course. did you?', 'why not?', 'of course.',
       'what do you mean?', 'what do you mean?', 'what do you mean?',
       'what do you mean?', 'what do you mean?', 'me too.',
       'what do you mean?', 'why not?',
       'i think that she is very pretty.', 'why not?',
       'they say that we are funny-looking.', 'yes, i did.', 'why not?',
       'i need a notebook.', 'yes, i do.', 'of course.',
       'what do you mean?', 'what do you mean?',
       'but typing is causing you pain.', 'why not?', 'of course.',
       'w

In [13]:
# Function to get response from the trained model
def get_response(user_message, pipeline):
    response = pipeline.predict([user_message])[0]
    logging.debug(f"User message: {user_message}, Bot response: {response}")
    return response


In [14]:
# Function to initialize and start the chat
def start_chat(pipeline):
    # Create the main window
    chat_window = tk.Tk()
    chat_window.title("Chat-o-fy")
    chat_window.configure(bg='light blue')

    # Function to send a message
    def send_message():
        user_message = user_input.get("1.0", tk.END).strip()
        if user_message == "":
            messagebox.showwarning("Empty Message", "Please type a message.")
            return
        
        user_input.delete("1.0", tk.END)
        add_user_message(user_message)
        bot_response = get_response(user_message, pipeline)
        add_bot_message(bot_response)

    # Function to add user message to chat history
    def add_user_message(message):
        chat_history.config(state=tk.NORMAL)
        chat_history.insert(tk.END, "\n")
        chat_history.image_create(tk.END, image=you_img)
        chat_history.insert(tk.END, " You: " + message + "\n", "user")
        chat_history.config(state=tk.DISABLED)
        chat_history.yview(tk.END)

    # Function to add bot message to chat history
    def add_bot_message(message):
        chat_history.config(state=tk.NORMAL)
        chat_history.insert(tk.END, "\n")
        chat_history.image_create(tk.END, image=robot_head_img)
        chat_history.insert(tk.END, " Chat-o-fy: " + message + "\n", "bot")
        chat_history.config(state=tk.DISABLED)
        chat_history.yview(tk.END)

    # Create and place widgets
    chat_history = scrolledtext.ScrolledText(chat_window, wrap=tk.WORD, width=50, height=20, bg="white", fg="black", font=("Arial", 12), bd=1, relief="solid")
    chat_history.tag_config("user", justify='right', background="white", foreground="black", borderwidth=2, relief="solid")
    chat_history.tag_config("bot", justify='left', background="white", foreground="black", borderwidth=2, relief="solid")
    chat_history.config(state=tk.DISABLED)
    chat_history.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

    user_input = tk.Text(chat_window, wrap=tk.WORD, width=40, height=3, bg="white", fg="black", font=("Arial", 12), bd=1, relief="solid")
    user_input.grid(row=1, column=0, padx=10, pady=10)

    send_button = tk.Button(chat_window, text="Send", command=send_message, bg="light blue", fg="black", font=("Arial", 12, "bold"), bd=1, relief="solid")
    send_button.grid(row=1, column=1, padx=10, pady=10)

    # Load robot head image and "You" placeholder image
    try:
        global robot_head_img, you_img
        robot_head_img = Image.open("robot_head.png")  # Update with correct path
        robot_head_img = robot_head_img.resize((40, 40), Image.LANCZOS)
        robot_head_img = ImageTk.PhotoImage(robot_head_img)
        
        # Placeholder image for "You"
        you_img = Image.new('RGBA', (40, 40), (255, 255, 255, 0))
        you_img_draw = ImageDraw.Draw(you_img)
        you_img_draw.text((10, 10), "You", fill="black")
        you_img = ImageTk.PhotoImage(you_img)
    except FileNotFoundError:
        messagebox.showerror("Image Not Found", "robot_head.png image not found.")
        robot_head_img = None
        you_img = None

    # Initial greeting message from bot
    add_bot_message("Hello! I am Chat-o-fy. How can I help you today?")

    chat_window.mainloop()

In [17]:
# Start the chat interface
start_chat(pipeline)

DEBUG:root:User message: hi, how are you doing?, Bot response: i'm doing great. what about you?
DEBUG:root:User message: i'm pretty good. thanks for asking., Bot response: no problem. so how have you been?
DEBUG:root:User message: i've been great. what about you?, Bot response: i've been good. i'm in school right now.
DEBUG:root:User message: what school do you go to?, Bot response: yes, i do.
DEBUG:root:User message: do you like it there?, Bot response: of course.
